In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.9 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=14d77fa99b536df53e808ae3c9b4df43153083501ada669208cc07ee3d5f6069
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [2]:
import pandas as pd
from pyspark.sql.functions import col, explode


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
from pyspark.sql import SparkSession
#spark.stop()
spark = SparkSession \
.builder \
.config("spark.executor.instances", "1")\
.config('spark.driver.memory','1g')\
.config('spark.executor.memory', '1g') \
.getOrCreate()

In [5]:
df = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/melodice.csv', sep=',', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()
df.show()

+-------+-------+-----+---+----+---+---+---+-----+---+---+-------+-----+--------------------+---------+-----------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|game_id|link_id|   EA| EE|  EL| EN| EP| RM|   SA| VD| VU|current|valid|          game_title|game_year|game_playing_time|game_rating|game_voters|           game_desc|           game_tags|          link_title|            link_url|       link_duration|
+-------+-------+-----+---+----+---+---+---+-----+---+---+-------+-----+--------------------+---------+-----------------+-----------+-----------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  15504|    445|  3.0|0.0| 0.0|0.0|0.0|0.0|  3.0|0.0|0.0|      0|    0|         Goblin King|     1996|              180|        6.8|         20|"In ""Goblin King...| with their small...| come down from t...|               towns|             castl

In [5]:
df = spark.read.csv('/content/gdrive/MyDrive/musicrecom/melodice/melodice_all_events.csv', sep=',', inferSchema=True, header=True, nullValue='NA', nanValue='NA',emptyValue='NA').dropna()


In [6]:
df.show()

+-------+-------+---+--------+--------------------+----------+
|game_id|link_id|idx|cookieid|          event_date|event_type|
+-------+-------+---+--------+--------------------+----------+
|    242|   3260| -1|     0-0|2016-05-19 18:12:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 18:18:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 18:21:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 18:50:...|        EA|
|    242|   3267| -1|     0-0|2016-05-19 18:56:...|        EA|
|    242|   3260| -1|     0-0|2016-05-19 18:59:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 19:04:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 19:05:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 19:06:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 19:08:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 19:13:...|    

In [7]:
from pyspark.sql.functions import countDistinct


In [8]:
df.select(countDistinct("cookieid")).show()

+------------------------+
|count(DISTINCT cookieid)|
+------------------------+
|                 5505306|
+------------------------+



In [9]:
df.count()

73245966

In [10]:
listened = df.filter(df.event_type == "EA")
listened.show()

+-------+-------+---+--------+--------------------+----------+
|game_id|link_id|idx|cookieid|          event_date|event_type|
+-------+-------+---+--------+--------------------+----------+
|    242|   3260| -1|     0-0|2016-05-19 18:12:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 18:18:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 18:19:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 18:21:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 18:50:...|        EA|
|    242|   3267| -1|     0-0|2016-05-19 18:56:...|        EA|
|    242|   3260| -1|     0-0|2016-05-19 18:59:...|        EA|
|    242|   3261| -1|     0-0|2016-05-19 19:04:...|        EA|
|    242|   3262| -1|     0-0|2016-05-19 19:05:...|        EA|
|    242|   3263| -1|     0-0|2016-05-19 19:06:...|        EA|
|    242|   3264| -1|     0-0|2016-05-19 19:08:...|        EA|
|    242|   3265| -1|     0-0|2016-05-19 19:13:...|    

In [11]:
listened.count()

30504155

In [12]:
upvoted = df.filter(df.event_type == "VU")
upvoted.count()

26716

In [13]:
downvoted = df.filter(df.event_type == "VD")
downvoted.count()

17190

In [14]:
skipped = df.filter((df.event_type == "EP") | (df.event_type == "EN") | (df.event_type == "EL"))
skipped.count()

2483347

In [15]:
from pyspark.sql.functions import countDistinct, count


In [16]:
user_song = listened.groupBy("link_id", "cookieid").count()

In [17]:
user_song = user_song.withColumnRenamed("count", "listening_count")

In [19]:
user_listenings = user_song.groupBy("cookieid").sum("listening_count")

In [20]:
user_listenings.show()

+---------------+--------------------+
|       cookieid|sum(listening_count)|
+---------------+--------------------+
|1482655249-4433|                  28|
|1577289829-8080|                  76|
|1485374646-1865|                  52|
|1488726866-5999|                   1|
|1500075691-1706|                 100|
|1616403835-1792|                1788|
|1523575784-2265|                  18|
|1577978010-4841|                   8|
|1622290810-4476|                  48|
|1628959544-9753|                 346|
|1485629330-7160|                  20|
|1598879289-9764|                  37|
|1480717233-1209|                  65|
|1505584045-1276|                  21|
|1659698672-1089|                   1|
|1618766125-5824|                  25|
|1546742461-7896|                1039|
|1663867712-8142|                  38|
|1612868252-6257|                  31|
|1478969379-1953|                 399|
+---------------+--------------------+
only showing top 20 rows



In [23]:
from pyspark.sql.functions import mean

user_listenings.select(mean("sum(listening_count)")).show()


+-------------------------+
|avg(sum(listening_count))|
+-------------------------+
|       18.772989163614163|
+-------------------------+



In [ ]:
# test = spark.read.csv("user_song.csv", header='true')
# test.show()